In [10]:
import redis
import pandas as pd
from pyspark.sql import SparkSession
from elasticsearch import Elasticsearch
import pyspark.sql.functions as psf
import pyspark.sql.types as pst


# run news_to_redis !!

connection = redis.StrictRedis(
    host='localhost',  # Replace with your Redis server host
    port=6379,         # Default Redis port
    db=0,              # Default database
    decode_responses=True  # Ensures responses are returned as strings
)


data = {"source":[], "min_sentiment":[], "max_sentiment": [], "avg_sentiment": [], "std_sentiment": []}
keys = []
# iterate over all saved news aggregates
for key in connection.scan_iter("sentiments_aggregates:*"):
    keys.append(key)
    fields = connection.hkeys(key)
    for field in data.keys():
        if field in fields:
            data[field].append(connection.hget(key, field))
        else:
            data[field].append(None)

df = pd.DataFrame(data)
df["key"] = keys
connection.close()
df

,source,min_sentiment,max_sentiment,avg_sentiment,std_sentiment,key
0,Cryptocurrency News,0.6588,0.6588,0.6588,None,sentiments_aggregates:9186c9573b164ce9859192bd...
1,Cointelegraph,-0.2023,0.6597,0.009493421052631578,0.2745888593688274,sentiments_aggregates:bd17aefb4f9b432b92254d8f...
2,CoinDesk,-0.0772,0.6602,0.06357894736842107,0.31024854297974175,sentiments_aggregates:8b161887a05b479190225752...


In [5]:
# run logs_to_elasticsearch

es = Elasticsearch(
    hosts=["http://localhost:9200"],
)



In [97]:
# Start a scroll query
response = es.search(
    index="tickers",
    query={"match_all": {}},
    size=100,
    scroll="1m"  # Keep the search context alive for 1 minute
)

scroll_id = response["_scroll_id"]

data = []
while True:
    # Fetch the next batch of documents
    response = es.scroll(scroll_id=scroll_id, scroll="1m")
    
    hits = response["hits"]["hits"]
    if not hits:
        break  # Exit when no more documents are found
    
    for hit in hits:
        data.append(hit['_source'])

    scroll_id = response["_scroll_id"]

# Clear the scroll context when done
es.clear_scroll(scroll_id=scroll_id)

df = pd.DataFrame(data)
df.describe()

,price,open_24h,volume_24h,low_24h,high_24h,volume_30d,best_bid,best_bid_size,best_ask,best_ask_size,trade_id,last_size,date,timestamp
count,40877.000000,40877.000000,40877.000000,40877.000000,40877.000000,40877.000000,40877.000000,4.087700e+04,40877.000000,4.087700e+04,4.087700e+04,4.087700e+04,4.087700e+04,4.087700e+04
mean,98506.807279,98259.989388,8971.079319,97746.880334,100235.576467,431979.158848,98505.445436,9.507203e-02,98507.808130,1.282163e-01,7.457042e+08,4.878070e-03,1.734972e+12,1.735028e+12
std,835.769201,593.214932,4463.253918,429.094430,1537.853069,88006.550128,835.940173,1.491669e-01,835.894191,4.871345e-01,9.181414e+06,3.709513e-02,1.187025e+09,1.194702e+09
min,97550.470000,97714.250000,4698.150348,97516.650000,98969.920000,359404.266655,97550.470000,1.000000e-08,97559.160000,8.000000e-08,7.345152e+08,1.000000e-08,1.733530e+12,1.733570e+12
25%,97785.560000,97804.670000,5329.328852,97516.650000,98969.920000,359542.173125,97783.940000,1.359340e-03,97786.170000,3.340390e-03,7.345624e+08,1.603000e-05,1.733530e+12,1.733579e+12
50%,98003.210000,97976.460000,5811.546324,97516.650000,98969.920000,359616.091494,98001.470000,3.225660e-02,98003.250000,3.674168e-02,7.532488e+08,1.516600e-04,1.735949e+12,1.736009e+12
75%,99476.310000,98489.440000,14534.043147,97581.440000,102104.120000,538838.056939,99475.940000,1.157300e-01,99477.860000,1.921731e-01,7.532635e+08,1.573150e-03,1.735949e+12,1.736012e+12
max,99795.520000,99581.580000,15253.266547,98666.260000,102104.120000,539111.734722,99784.010000,1.490210e+00,99795.520000,1.289041e+01,7.532782e+08,4.847932e+00,1.735949e+12,1.736015e+12


In [111]:
from datetime import datetime
min_timestamp = df["timestamp"].min()
max_timestamp = df["timestamp"].max()
print("tickers min timestamp: ", min_timestamp)
print("tickers max timestamp: ", max_timestamp)

tickers min timestamp:  1733569661083
tickers max timestamp:  1736015065228


In [126]:
sdf = spark.read.parquet("hdfs://localhost/crypto/exchange/ticker")

hdfs_count = sdf.filter(psf.to_timestamp(psf.col("time")) <= datetime.fromtimestamp(max_timestamp / 1000.0)).filter(psf.to_timestamp(psf.col("time")) >= datetime.fromtimestamp(min_timestamp / 1000.0)).count()
print("ticker elastic count: ", df.shape[0])
print("ticker hdfs count: ", hdfs_count)

ticker elastic count:  40877
ticker hdfs count:  35999


In [68]:
# Start a scroll query
response = es.search(
    index="transactions",
    query={"match_all": {}},
    size=100,
    scroll="1m"  # Keep the search context alive for 1 minute
)

scroll_id = response["_scroll_id"]

data = []
while True:
    # Fetch the next batch of documents
    response = es.scroll(scroll_id=scroll_id, scroll="1m")
    
    hits = response["hits"]["hits"]
    if not hits:
        break  # Exit when no more documents are found
    
    for hit in hits:
        data.append(hit['_source'])

    scroll_id = response["_scroll_id"]

# Clear the scroll context when done
es.clear_scroll(scroll_id=scroll_id)

edf = pd.DataFrame(data)
edf.describe()

,timestamp
count,5.817000e+03
mean,1.736012e+12
std,2.001078e+06
min,1.736008e+12
25%,1.736010e+12
50%,1.736012e+12
75%,1.736013e+12
max,1.736015e+12


In [87]:
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5817 entries, 0 to 5816
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   op          5817 non-null   object
 1   time        5817 non-null   object
 2   hash        5817 non-null   object
 3   relayed_by  5817 non-null   object
 4   event       5817 non-null   object
 5   timestamp   5817 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 272.8+ KB


In [91]:
edf.size

34902

In [92]:
print("transactions max time: ", edf["time"].max())
print("transactions min time: ", edf["time"].min())
min_timestamp = edf["timestamp"].min()
max_timestamp = edf["timestamp"].max()
print("transactions max timestamp: ", max_timestamp)
print("transactions min timestamp: ", min_timestamp)

transactions max time:  2025-01-04 18:26:03
transactions min time:  2025-01-04 16:30:47
transactions max timestamp:  1736015163000
transactions min timestamp:  1736008247000


In [93]:
spark = SparkSession.builder.getOrCreate()


df = spark.read.parquet("hdfs://localhost/crypto/blockchain/transactions")
print("transaction count elastic: ", edf.size)
print("transaction count hdfs: ", df.filter(psf.col("x.time") >= min_timestamp // 1000).filter(psf.col("x.time") <= max_timestamp // 1000).count())

transaction count elastic:  34902


transaction count hdfs:  45935


In [128]:
# calculate training dataset !
training_df = spark.read.parquet("hdfs://localhost/user/hive/warehouse/training_data.db/full_training_dataset")
training_df.count()

35999

In [ ]:
training_df.select([
    psf.sum(psf.col(c).isNull().cast("int")).alias(c) for c in training_df.columns
]).show(vertical=True)

-RECORD 0--------------------------------
 price                             | 0   
 open_24h                          | 0   
 volume_24h                        | 0   
 low_24h                           | 0   
 high_24h                          | 0   
 volume_30d                        | 0   
 best_bid                          | 0   
 best_bid_size                     | 0   
 best_ask                          | 0   
 best_ask_size                     | 0   
 avg_price_last_15s                | 0   
 avg_price_last_30s                | 0   
 avg_price_last_60s                | 0   
 transaction_count_last_15s        | 0   
 transaction_count_last_30s        | 0   
 transaction_count_last_60s        | 0   
 coindesk_avg_sentiment            | 0   
 coindesk_std_sentiment            | 0   
 coindesk_max_sentiment            | 0   
 coindesk_min_sentiment            | 0   
 cointelegraph_avg_sentiment       | 0   
 cointelegraph_std_sentiment       | 0   
 cointelegraph_max_sentiment      

25/01/04 23:47:32 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1219)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java

In [3]:
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# start online ml

model = LogisticRegressionModel.load("hdfs://localhost/user/hadoop/model.model/")
model

LogisticRegressionModel: uid=LogisticRegression_a06db5eca165, numClasses=2, numFeatures=28

In [20]:
response = es.search(
    index="predictions",
    query={"match_all": {}},
    size=1000,
)

In [21]:
df = pd.DataFrame([x["_source"] for x in response["hits"]["hits"]])
df

,price,open_24h,volume_24h,low_24h,high_24h,volume_30d,best_bid,best_bid_size,best_ask,best_ask_size,...,cointelegraph_std_sentiment,cointelegraph_max_sentiment,cointelegraph_min_sentiment,cryptocurrency_news_avg_sentiment,cryptocurrency_news_std_sentiment,cryptocurrency_news_max_sentiment,cryptocurrency_news_min_sentiment,timestamp,event,prediction
0,97574.10,97853.76,2257.644636,97516.65,98761.02,343263.294827,97574.09,0.025027,97574.10,0.194815,...,0.347298,0.3182,-0.7125,0.0,0.0,0.0,0.0,1736079018393,ticker,0.0
1,97574.10,97853.76,2257.644638,97516.65,98761.02,343263.294828,97574.09,0.025027,97574.10,0.251814,...,0.347298,0.3182,-0.7125,0.0,0.0,0.0,0.0,1736079018629,ticker,0.0
2,97574.10,97853.76,2257.644643,97516.65,98761.02,343263.294833,97574.09,0.025027,97574.10,0.251809,...,0.347298,0.3182,-0.7125,0.0,0.0,0.0,0.0,1736079019682,ticker,0.0
3,97574.10,97853.76,2257.644643,97516.65,98761.02,343263.294834,97574.09,0.025027,97574.10,0.251808,...,0.347298,0.3182,-0.7125,0.0,0.0,0.0,0.0,1736079019833,ticker,0.0
4,97574.10,97853.76,2257.644694,97516.65,98761.02,343263.294885,97574.09,0.025027,97574.10,0.251757,...,0.347298,0.3182,-0.7125,0.0,0.0,0.0,0.0,1736079019961,ticker,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,97703.74,97853.76,2263.656472,97516.65,98761.02,343269.306663,97703.73,0.451321,97703.74,0.011594,...,0.347298,0.3182,-0.7125,0.0,0.0,0.0,0.0,1736079362947,ticker,1.0
996,97703.74,97853.76,2263.668066,97516.65,98761.02,343269.318256,97703.73,0.451721,97704.54,0.004880,...,0.347298,0.3182,-0.7125,0.0,0.0,0.0,0.0,1736079363178,ticker,1.0
997,97708.40,97853.76,2263.672960,97516.65,98761.02,343269.323150,97708.39,0.004995,97708.40,0.000220,...,0.347298,0.3182,-0.7125,0.0,0.0,0.0,0.0,1736079363343,ticker,1.0
998,97708.39,97853.76,2263.672951,97516.65,98761.02,343269.323141,97708.39,0.004995,97708.40,0.000229,...,0.347298,0.3182,-0.7125,0.0,0.0,0.0,0.0,1736079363317,ticker,1.0


In [22]:
df["prediction"].max()

np.float64(1.0)

In [23]:
df["prediction"].min()

np.float64(0.0)